# The Random Sampling with Replacement

Assumption: 
- Assume each number got selected at a equal weights (probability)
- Assume we will not update the weight list

quite easy to implement


# The Random Sampling without Replacement
- Designed to generate unique random numbers within a specified range (from 0 to n). Each call to the getRand method returns a random number that hasn't been returned before, ensuring that all numbers in the range are returned exactly once in a random order.
- This is similar to shuffling a deck of cards and drawing one card at a time without replacement.

Assumption: 
- Assume each number got selected at a equal weights (probability)
- Assume we will not update the weight list

Requirement:
- **get_random_without_replacement** must be O(1)

Related Leetcode:
- https://leetcode.com/problems/insert-delete-getrandom-o1/
- https://leetcode.com/problems/random-pick-with-weight/description/
- 2.k种不同颜色的球，个数为[n1, n2, n3, ..., n_k]， 如何sample without replacement？要求sample operation time complexity log(k)

# Naive Pseudocode Code:

```python

candidate = {0, 1, 2, 3, 4, 5, 6} # space: O(n)
Loop:
    candidate_list = list(candidate) # time: O(n) <-------- bottleneck
    random_num = random.choice(candidate_list) # time: O(1)
    candidate.remove(random_num) # time: O(1)

here we maintain a set of num and we convert the set to list on the fly.
```

Since we do this conversion on the fly, this is not efficient.

Trick: for candidate list, swamp the target number, and last item inside the candidate list,
and pop the last item from the candidate list. 

However, swamp trick is not free, in order to swamp in a array, we need index of the num.
We can easily get the last item index, but no way we get index of a random number at o(1) time.
we can only achieve this with a hashmap where {num: num_idx}


# Improved Pseudocode Code:
```python
candidate_list = [0, 1, 2, 3, 4, 5, 6]
idx_by_num = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6}
Loop:
    random_num = random.choice(candidate_list) # time: O(1)

    # remove from list with swamp. For swamping, we need index information:
    rand_idx = idx_by_num[random_num]

    last_val = candidate_list[-1]
    candidate_list[rand_idx], candidate_list[-1] = candidate_list[-1], candidate_list[rand_idx] # time: O(1)
    candidate_list.pop() # time: O(1)

    # we need to update idx_by_num since 6 is no longer at position=6 anymore
    idx_by_num[last_val] = rand_idx
    idx_by_num.pop(random_num) # time: O(1)
    
    return random_num
```

In [ ]:
import random

class RandomNumberGenerator:
    """ without replacement

    eg. random_num = 4
    step1:
        [0, 1, 2, 3, 4, 5, 6]
    step2: list: swamp
        [0, 1, 2, 3, 6, 5, 4]
    step3: list: remove last item
        [0, 1, 2, 3, 6, 5]
    """

    def __init__(self, n):
        self.n = n
        self.nums = [num for num in range(n)]
        self.idx_by_num = {num: num for num in range(n)}
        
    def get_random_without_replacement(self):
        rand_num = random.choice(self.nums) # time: O(1)

        # swamp: last item and rand_num
        last_val = self.nums[-1]
        rand_idx = self.idx_by_num[rand_num]
        self.nums[rand_idx], self.nums[-1] = self.nums[-1], self.nums[rand_idx] # time: O(1)

        # update idx_by_num
        self.idx_by_num[last_val] = rand_idx # time: O(1)

        # remove rand_num for nums and idx_by_num
        self.nums.pop()  # time: O(1)
        self.idx_by_num.pop(rand_num)  # time: O(1)

        return rand_num



We can confirm that all the operation on the get_random_without_replacement are O(1). And in this case, get_random_without_replacement itself is O(1) operation.

To conclude: the entire purpose of dict mapping is that we need to store and index in order to swamp the random number and last number inside the list.

Notice that we used a list and a separate dict which tracking the idex. However, it's possible to optimize your original implementation even further to reduce space usage while still maintaining O(1) time complexity for the get_random_without_replacement method. The key lies in eliminating the need for an additional dictionary by leveraging the properties of the list itself.

# Optimized Implementation: Shuffle
Notes: Another idea to generate random number is that we can shuffle the array, and then go from left to the right to generate random number. In short, you can eliminate the dictionary by using shuffle. 

```python
class RandomNumGenerator:

    def __init__(self, n):
        self.n_list = [i for i in range(n)]
        shuffle(self.n_list) # assume shuffle in-place
        self.count = 0
    
    def get_random_without_replacement(self):
        rand_val = self.n_list[self.count]
        self.count += 1 
        return rand_val
    
```

# Fisher–Yates shuffle
This approach maintains a single list and uses index manipulation to ensure each number is selected exactly once in O(1) time per selection. -> https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle#Python_Implementation

Notes: to shuffle an array we cannot achieve an time complexity smaller than O(n) since we need to move the every current element to another places, which means we need to touch all the element inside the array.


```python
import random 

def shuffle(n: int) -> list[int]:
    """Non-in-placement shuffle"""
    numbers = list(range(n))
    shuffled = []
    while numbers:
        idx = random.randint(0, len(numbers) - 1)
        shuffled.append(numbers[idx])
        numbers.pop(idx)
    return shuffled
```

# Random Sampling with Weights with Replacement

What if we are give a list of number and a list of weights associated with that number?

To continue on the Fisher–Yates shuffle approach, we can simply add equvilant more number in the array and to Fisher–Yates shuffle. eg.nums = [0, 1, 2, 3], weights=[0.01, 0.2, 0.79] will require a number of 79 3 inserted into candidate array which is infeasible.

To random sampling a number with associated weight, we can leverage continuous cumulative distribution


```python
          A  B  C  D
weight = [2, 5, 3, 4] -> [2, 7, 10, 14]

{
    [0, 2]:  A,
    [2, 7]:  B,
    [7, 10]: C,
    [10,14]: D
}


```
Now we can generate a random integer from uniform distribution between 0 and 14, lets say 13, and we need to find which range it falls into. In order to find the right range, you will need to do a linear probe to compare the start and end for the range.

```python
Number Line: 0 ---------------------- 4
             |       |                |
             0       1                4
```


However, prefix_sum is always increasing, then We do  binary search log(k) to find the right range.



# Optimized Weighted Sampling with Replacement

- https://www.keithschwarz.com/darts-dice-coins/
- Alias method: https://leetcode.com/problems/random-pick-with-weight/solutions/671439/python-smart-o-1-solution-with-detailed-explanation/

# Random Sampling with Weights without Replacement

Alias Method is specifically designed for weighted sampling with replacement. It excels in scenarios where you need to perform numerous independent samples efficiently, maintaining constant-time O(1) performance for each selection. However, when it comes to weighted sampling without replacement, the Alias Method is not directly applicable. Let's delve deeper into this distinction and explore suitable alternatives for sampling without replacement.





# Random Sampling from Data Stream
- Reservoir Sampling
